# 用PaddlePaddle复现Fourier Neural Operator



## 训练流程（当前仍有bug）

下面的代码定义了一个简单的训练流程。使用官方代码给出的Darcy flow 数据集，我们尝试训练一个FNO模型。训练过程中，我们使用了Adam优化器。

## 当前的bug的定位

目前我们认为上述训练流程的错误基本上集中在`FactorizedFourierLayer`中。我们认为这个层的实现有问题，但是我们目前还没有找到问题的根源。我们希望能够得到一些帮助。

### 1. 只训练这一层后面的参数

我们发现，如果我们只训练这一层后面的参数，那么模型的训练效果会正常。这个现象表明，这一层的梯度计算是有问题的。